In [1]:
from pykalman import KalmanFilter
import numpy as np
import pandas as pd
from scipy import poly1d
from datetime import datetime
import yfinance as yf
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-darkgrid')
plt.rcParams['figure.figsize'] = (10,7)

In [2]:
cobrandingPairsDf = pd.read_csv('cobrandingPairs.csv')

In [3]:
cobrandingPairsDf = cobrandingPairsDf.drop('Unnamed: 0',1)

In [4]:
cobrandingPairsDf

,Stock1,Stock2
0,NVDA,TSM
1,BMWYY,LVMUY
2,NKE,AAPL
3,NFLX,AMZN
4,MSFT,AMZN
5,FB,AMZN
6,MSFT,EBAY
7,FB,AMZN
8,FB,MSFT
9,QCOM,TSM


In [5]:
dfSP = pd.read_csv('sp_data.csv')
dfSP = dfSP[:-1]

In [6]:
dfSP = dfSP.drop('Unnamed: 0',1)
dfSP = dfSP.drop('Unnamed: 506',1)

In [7]:
dfSP

,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,...,WYNN,XEL,XLNX,XYL,YUM,ZBRA,ZBH,ZION,ZTS,Date
0,166.347397,73.888474,70.061684,260.269989,169.237976,46.289951,276.769989,27.090000,157.008972,15.022331,...,125.465141,54.252647,113.940872,76.228203,97.850586,195.889999,120.165443,45.380920,101.123032,2019-05-10 00:00:00
1,163.264282,74.082359,69.989822,261.940002,169.861115,46.111988,275.790009,27.209999,158.266037,15.153941,...,117.718361,54.793182,107.820885,74.882660,96.866585,181.009995,116.904419,43.123123,100.193748,2019-05-13 00:00:00
2,163.804535,74.111435,69.576584,258.920013,169.705307,45.499008,278.480011,27.959999,156.514053,15.276150,...,119.118965,54.347469,110.296173,73.624863,97.127052,186.720001,116.993073,43.507229,100.094894,2019-05-14 00:00:00
3,162.137253,73.190506,69.055550,252.369995,166.511795,42.918560,267.700012,26.240000,153.791946,15.144540,...,120.983215,54.262131,111.899628,74.541389,97.223923,185.179993,116.796043,42.554863,100.678154,2019-05-15 00:00:00
4,163.199097,73.704277,70.465942,256.690002,169.033508,44.193954,271.859985,27.320000,154.168106,15.116338,...,122.973030,54.745762,103.765022,74.356125,97.717987,190.020004,116.333000,42.969749,101.755714,2019-05-16 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,198.610001,119.529999,114.680000,308.640015,292.170013,91.150002,503.459991,78.550003,201.860001,27.350000,...,127.809998,70.750000,122.029999,117.360001,121.089996,482.420013,173.550003,58.220001,173.860001,2021-05-05 00:00:00
501,199.380005,117.540001,113.900002,305.000000,291.600006,88.690002,490.700012,78.610001,204.149994,26.709999,...,122.779999,71.550003,122.139999,118.750000,121.290001,471.410004,173.229996,59.180000,167.520004,2021-05-06 00:00:00
502,201.440002,118.510002,115.779999,300.790009,289.519989,90.080002,486.690002,77.830002,202.809998,26.469999,...,126.290001,71.320000,124.430000,119.339996,121.230003,488.579987,172.490005,59.270000,171.550003,2021-05-07 00:00:00
503,202.410004,118.790001,116.080002,295.720001,291.160004,93.029999,483.609985,77.889999,204.160004,25.450001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-07 00:00:00


In [16]:
###### Arg: dataframe of stock pairs, 
##Returns: return after trading in those positions 
def getTradingSignals(df):
    returns = []
    removeIndex = []
    for index,stocks in df.iterrows():
        stock1 = stocks['Stock1']
        stock2 = stocks['Stock2']
        kalmanDf = pd.DataFrame()

        pair1 = yf.Ticker(stock1)
        pair2 = yf.Ticker(stock2)

        pair1 = pair1.history(period = '2y')
        pair2 = pair2.history(period = '2y')
        if(pair1.shape != pair2.shape):
            removeIndex.append(index)
            continue
        
        ##Read from CSV (2013 - 2018)
        tempStock1Df = pd.read_csv('all_stocks_5yr.csv')
        tempStock1Df = tempStock1Df[['date', 'close','Name']]
        tempStock1Df.columns = ['Date', 'Close','Name']
        tempStock1Df = tempStock1Df[tempStock1Df['Name']==stock1]
        tempStock1Df = tempStock1Df.drop('Name',1)
        tempStock1Df.columns = ['Date', stock1]
        
        tempStock2Df = pd.read_csv('all_stocks_5yr.csv')
        tempStock2Df = tempStock2Df[['date', 'close','Name']]
        tempStock2Df.columns = ['Date', 'Close','Name']
        tempStock2Df = tempStock2Df[tempStock2Df['Name']==stock2]
        tempStock2Df = tempStock2Df.drop('Name',1)
        tempStock2Df.columns = ['Date', stock2]
        
        tempPairDf = tempStock1Df.merge(tempStock2Df, how='inner', on='Date')
        tempPairDf['Date']= pd.to_datetime(tempPairDf['Date'])
        flag = 0
        if(len(tempPairDf) > 0):
            flag = 1
#             print(tempPairDf.columns)
        
        kalmanDf[stock1] = pair1['Close'].values.astype(float)
        kalmanDf[stock2] = pair2['Close'].values.astype(float)
        kalmanDf['Date'] = pair1['Close'].index
        
        if(flag == 1):
#             print("here")
            kalmanDf = pd.concat([kalmanDf, tempPairDf])
#             print(kalmanDf)
#             kalmanDf = kalmanDf.join(tempPairDf, how='outer', on='Date')
        
#         print("finallen:",len(kalmanDf))
        kalmanDf = kalmanDf.sort_values(by='Date')

        kalmanDf['ratio'] = kalmanDf[stock1]/kalmanDf[stock2] 
        kf = KalmanFilter(transition_matrices = [1],
                  observation_matrices = [1],
                  initial_state_mean = 0,
                  initial_state_covariance = 1,
                  observation_covariance=1,
                  transition_covariance=.0001)
        mean, cov = kf.filter(kalmanDf['ratio'].values)
        kalmanDf['mean'] = mean.squeeze()
        kalmanDf['cov'] = cov.squeeze()
        kalmanDf['std'] = np.sqrt(kalmanDf['cov'])
        kalmanDf = kalmanDf.dropna()
        kalmanDf['ma'] = kalmanDf['ratio'].rolling(5).mean()
        kalmanDf['Zscore'] = (kalmanDf['ma'] - kalmanDf['mean'])/kalmanDf['std']

        kalmanDf['Buy'] = np.nan
        kalmanDf['Sell'] = np.nan

        for i in range (kalmanDf.shape[0]):
            if kalmanDf['Zscore'].iloc[i] < -1:
                kalmanDf['Buy'].iloc[i] = 1
                kalmanDf['Sell'].iloc[i] = -round(kalmanDf['ratio'].iloc[i],0)
            if kalmanDf['Zscore'].iloc[i] > 1:
                kalmanDf['Buy'].iloc[i] = -1
                kalmanDf['Sell'].iloc[i] = round(kalmanDf['ratio'].iloc[i],0)
            if (abs(kalmanDf['Zscore'].iloc[i]) < 1) & (abs(kalmanDf['Zscore'].iloc[i]) > 0):
                kalmanDf['Buy'].iloc[i] = 0
                kalmanDf['Sell'].iloc[i] = 0
        # Calculate returns
        
        kalmanDf['returns'] = ((kalmanDf[stock1]-kalmanDf[stock1].shift(1))/kalmanDf[stock1].shift(1))*kalmanDf['Buy'].shift(1)+ ((kalmanDf[stock2]-kalmanDf[stock2].shift(1))/kalmanDf[stock2].shift(1))*kalmanDf['Sell'].shift(1)
        kalmanDf['returns'].sum()
        kalmanDf.to_csv("NewResults/signals_MA5/Sectors/"+stock1+"_"+stock2+".csv")
        returns.append(kalmanDf['returns'].sum())
    return returns, removeIndex


In [9]:
returns, removeIndex = getTradingSignals(cobrandingPairsDf)
print(len(returns))
cobrandingPairsDf = cobrandingPairsDf.drop(removeIndex)
cobrandingPairsDf['returns'] = returns

In [10]:
cobrandingPairsDf

In [11]:
cobrandingPairsDf.to_csv("NewResults/signals_MA5/cobrandingPairsDf.csv")

In [67]:
# kmeansCointPairsDf = pd.read_csv('finalPairs.csv')
# kmeansCointPairsDf = kmeansCointPairsDf.drop('Unnamed: 0',1)

In [68]:
# kmeansCointPairsDf

In [69]:
# returns,removeIndex = getTradingSignals(kmeansCointPairsDf)
# print(len(returns))
# kmeansCointPairsDf['returns'] = returns

In [70]:
# kmeansCointPairsDf

In [263]:
kmeansCointCorrPairsDf = pd.read_csv('corrCointPairs.csv')
kmeansCointCorrPairsDf = kmeansCointCorrPairsDf.drop('Unnamed: 0',1)

In [264]:
returns, removeIndex = getTradingSignals(kmeansCointCorrPairsDf)
print(len(returns))

/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

...

/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

1263


In [265]:
len(kmeansCointCorrPairsDf)

1344

In [266]:
kmeansCointCorrPairsDf = kmeansCointCorrPairsDf.drop(removeIndex)

In [267]:
kmeansCointCorrPairsDf['returns'] = returns

In [268]:
kmeansCointCorrPairsDf.sort_values(by = 'returns', ascending = False)

,Stock1,Stock2,returns
1068,IQV,NWS,14.206661
1067,IQV,NWSA,12.372615
1060,IPGP,PENN,11.588099
1150,LYB,RF,11.523926
282,AMP,NWSA,10.640134
...,...,...,...
255,AXP,IVZ,-5.021199
868,IT,IVZ,-6.454435
281,AMP,FCX,-8.504959
99,ISRG,TSLA,-9.239697


In [271]:
kmeansCointCorrPairsDf.to_csv("NewResults/signals_MA5/kmeansCointCorrPairsDf.csv")

In [273]:
###Sectorwise pairs
sectorDf = pd.read_csv("sec_wise_copaird.csv")
sectorDf = sectorDf.drop('Unnamed: 0',1)
sectorDf

,Consumer Discretionary,Health Care,Communication Services,Utilities,Consumer Staples,Materials,Industrials,Financials,Information Technology,Energy,Real Estate
0,"('SBUX', 'VFC')","('IQV', 'LLY')","('CMCSA', 'DIS')","('ES', 'WEC')","('EL', 'KO')","('FCX', 'MLM')","('GD', 'PH')","('C', 'CMA')","('FFIV', 'V')","('CVX', 'MRO')","('ARE', 'EQR')"
1,"('EBAY', 'LEN')","('BDX', 'MDT')","('CMCSA', 'EA')","('DTE', 'EIX')","('CLX', 'WBA')","('FCX', 'VMC')","('CAT', 'RHI')","('AMP', 'AXP')","('AVGO', 'CTSH')","('APA', 'OXY')","('CCI', 'ESS')"
2,"('EXPE', 'ULTA')","('BDX', 'CVS')","('CHTR', 'EA')","('CMS', 'XEL')","('K', 'SJM')","('FCX', 'SEE')","('CAT', 'UAL')","('LNC', 'RF')","('FFIV', 'STX')","('HES', 'OKE')","('CCI', 'FRT')"


In [274]:
sectors = sectorDf.columns

In [12]:
##Consumer Discretionary
CommunicationServicesDf = pd.DataFrame([['FB','NWSA'],['FOXA','GOOG']], columns = ['Stock1', 'Stock2'])
ConsumerDiscretionaryDf = pd.DataFrame([['MCD','PHM'],['AMZN','LKQ']], columns = ['Stock1', 'Stock2'])
ConsumerStaplesDf = pd.DataFrame([['MDLZ', 'MNST'],['STZ', 'WBA']], columns = ['Stock1', 'Stock2'])
IndustrialsDf = pd.DataFrame([['GD', 'PH'],['CHRW', 'PWR']], columns = ['Stock1', 'Stock2'])
FinancialsDf = pd.DataFrame([['C', 'PRU'],['C', 'RF'],['C', 'USB']], columns = ['Stock1', 'Stock2'])
InformationTechnologyDf = pd.DataFrame([['FFIV', 'V'],['AVGO', 'CTSH'],['AVGO', 'GPN']], columns = ['Stock1', 'Stock2'])
MaterialsDf = pd.DataFrame([['FCX', 'NEM'],['LYB', 'NEM']], columns = ['Stock1', 'Stock2'])
HealthCareDf = pd.DataFrame([['CNC', 'MTD'],['REGN', 'UNH'],['HCA', 'UHS']], columns = ['Stock1', 'Stock2'])
RealEstateDf = pd.DataFrame([['CCI', 'UDR'] ,['ARE', 'EQR']], columns = ['Stock1', 'Stock2'])
UtilitiesDf = pd.DataFrame([['ES', 'WEC'],['AES', 'NRG']], columns = ['Stock1', 'Stock2'])

# EnergyDf = pd.DataFrame([['CVX','MRO'],['APA','OXY'],['HES','OKE']], columns = ['Stock1', 'Stock2'])


# sectors = ['ConsumerDiscretionaryDf','HealthCareDf','CommunicationServicesDf','UtilitiesDf','ConsumerStaplesDf',
#            'MaterialsDf','IndustrialsDf','FinancialsDf','InformationTechnologyDf','EnergyDf','RealEstateDf']

In [13]:
sectorDf = pd.concat([ConsumerDiscretionaryDf, HealthCareDf, CommunicationServicesDf, UtilitiesDf,ConsumerStaplesDf,
                     MaterialsDf,IndustrialsDf, FinancialsDf, InformationTechnologyDf, RealEstateDf ])
sectorDf

,Stock1,Stock2
0,MCD,PHM
1,AMZN,LKQ
0,CNC,MTD
1,REGN,UNH
2,HCA,UHS
0,FB,NWSA
1,FOXA,GOOG
0,ES,WEC
1,AES,NRG
0,MDLZ,MNST


In [14]:
sectorDf = sectorDf.reset_index(drop=True)
sectorDf

,Stock1,Stock2
0,MCD,PHM
1,AMZN,LKQ
2,CNC,MTD
3,REGN,UNH
4,HCA,UHS
5,FB,NWSA
6,FOXA,GOOG
7,ES,WEC
8,AES,NRG
9,MDLZ,MNST


In [17]:
returns, removeIndex = getTradingSignals(sectorDf)

/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [18]:
print(len(sectorDf))
print(len(returns))

23
23


In [19]:
removeIndex

[]

In [21]:
# sectorDf1 = sectorDf.drop(removeIndex)
# sectorDf1

In [22]:
sectorDf['returns'] = returns

In [23]:
sectorDf

,Stock1,Stock2,returns
0,MCD,PHM,6.526504
1,AMZN,LKQ,55.639381
2,CNC,MTD,0.000000
3,REGN,UNH,0.702652
4,HCA,UHS,0.348471
5,FB,NWSA,11.942982
6,FOXA,GOOG,0.000000
7,ES,WEC,0.034295
8,AES,NRG,0.431741
9,MDLZ,MNST,0.480611


In [24]:
sectorDf.to_csv("NewResults/signals_MA5/sectorDF.csv")

In [26]:
sectorDf.sort_values(by = 'returns', ascending = False)

,Stock1,Stock2,returns
1,AMZN,LKQ,55.639381
5,FB,NWSA,11.942982
0,MCD,PHM,6.526504
19,AVGO,CTSH,5.062170
21,CCI,UDR,1.762726
20,AVGO,GPN,1.391603
11,FCX,NEM,0.944081
3,REGN,UNH,0.702652
14,CHRW,PWR,0.513728
9,MDLZ,MNST,0.480611
